<a href="https://colab.research.google.com/github/navi004/Deep-Learning-CSE4037-/blob/main/DL_Lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RNN - Lab A-5
* Naveen N (22MIA1049)  

In [6]:
# Install kagglehub
!pip install -q kagglehub

# Import kagglehub
import kagglehub

# Download the latest version of the IMDB dataset
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [7]:
import os

# Check the files in the path where the dataset is downloaded
print("Files in the dataset directory:")
os.listdir(path)

Files in the dataset directory:


['IMDB Dataset.csv']

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from sklearn.model_selection import train_test_split

In [11]:
data_file = os.path.join(path, 'IMDB Dataset.csv')
df = pd.read_csv(data_file)
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [12]:
t = Tokenizer(num_words=5000)

t.fit_on_texts(df['review'])

x = t.texts_to_sequences(df['review'])

max_length = 100

xp = pad_sequences(x, maxlen=max_length, padding = 'post')

In [13]:
y = np.array(df['sentiment'].map({'positive' : 1, 'negative' : 0}))
y

array([1, 1, 1, ..., 0, 0, 0])

In [14]:
x_train, x_test, y_train, y_test = train_test_split(xp, y, test_size=0.2, random_state=42)

print(x_train.shape)

(40000, 100)


In [15]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=32, input_length = max_length),
    SimpleRNN(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.6706 - loss: 0.5858 - val_accuracy: 0.7944 - val_loss: 0.3986
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.8305 - loss: 0.3591 - val_accuracy: 0.8317 - val_loss: 0.3868
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.8708 - loss: 0.3178 - val_accuracy: 0.8585 - val_loss: 0.3278
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.8934 - loss: 0.2696 - val_accuracy: 0.8572 - val_loss: 0.3426
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.9034 - loss: 0.2398 - val_accuracy: 0.8588 - val_loss: 0.3656


In [16]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8574 - loss: 0.3689
Test Loss: 0.3656357526779175, Test Accuracy: 0.8587999939918518


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (32, 100, 32)               │         160,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (32, 16)                    │             784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (32, 1)                     │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 482,405 (1.84 MB)

 Trainable params: 160,801 (628.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 321,604 (1.23 MB)

In [18]:
txt = 'The movie is fantastic but the story is poor'

txt = t.texts_to_sequences([txt])

txtP = pad_sequences(txt, maxlen=max_length, padding='post')

pred = model.predict(txtP)

print(pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
[[0.7944376]]


In [19]:
print("Positive" if pred[0][0] > 0.5 else "Negative")

Positive
